In [21]:
from datasets import load_dataset
import numpy as np

raw_dataset = load_dataset("csv", data_files="home/javilonso/NLP_Sentiment_UC3M/1.-DATA/dataset/Rest_Mex_2022_Sentiment_Analysis_Track_Train.csv", delimiter=";")

Using custom data configuration default-13c2fcae2abc0c4d
Reusing dataset csv (/home/javilonso/.cache/huggingface/datasets/csv/default-13c2fcae2abc0c4d/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
from transformers import pipeline

summarizer = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum")

In [31]:
#Data augmentation
## 1 * 4 steps
## 2 * 3 steps
## 3 * 2 steps

steps = {
    1: 5,
    2: 4,
    3: 3
}

print(raw_dataset)
#remove filter for whole dataset
# raw_dataset = raw_dataset.filter(lambda e, i: i<3, with_indices=True)
# print(raw_dataset)

def augment_data(examples):
    index = 0
    elements = examples["Opinion"].copy()
    for item in elements:
        print(index)
        if examples["Polarity"][index] == 1 or examples["Polarity"][index] == 2 or examples["Polarity"][index] == 3:
            for i in range(1,steps[examples["Polarity"][index]]):
                percentage = 100 - 20 * i
                lenght = 0
                print(len(item.split()))
                if len(item.split()) > 100:
                    lenght = 100
                else: 
                    lenght = len(item.split())
                output = summarizer(item, min_length=int(lenght * percentage / 100), max_length=lenght)
                examples["Title"].append(examples["Title"][index])
                examples["Opinion"].append(output[0]["summary_text"])
                examples["Polarity"].append(examples["Polarity"][index])
                examples["Attraction"].append(examples["Attraction"][index])
            index += 1
    return examples


DatasetDict({
    train: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction'],
        num_rows: 30212
    })
})


In [ ]:
raw_dataset = raw_dataset.map(augment_data, batched=True)

In [61]:
raw_dataset['train'][30226]

{'Title': 'Segunda vez allí',
 'Opinion': 'Esta visita de regreso era 365 días en la reserva. Mi marido y yo comimos aquí cuando estábamos de vacaciones en el mismo sitio el año pasado. Tuvimos una increíble comida año pasado que hemos estado muriendo a volver.\nAquí está lo que necesita saber:\nhacer una reserva. Esto lo hicimos esta vez. Aprendimos nuestra lección. Año pasado no tienen uno y esperamos una hora para conseguir una mesa. Este año nos sabía más y nos acomodaron enseguida (aunque llegamos después de una pareja que no tenía una reserva).\nPedimos el mismo año pasado (porque era lo que nos trajeron de nuevo, después de todo). Empezamos con el aperitivo de costilla y luego cada uno tenía una orden de las fajitas de Presidente (pollo, bistec y camarones). Un poco de todo. La salsa en las costillas es delicioso. Mi marido estaba rondan los nachos caseros (que te dan con salsa cuando estás sentado) en que tras las costillas habían desaparecido. Las tortillas que se sirven para 

In [72]:
datapd = raw_dataset["train"].to_pandas()

In [73]:
print(f'Polarity values({datapd.Polarity.count()}):\n{datapd.Polarity.value_counts()}\n')
print(f'Atraction values({datapd.Attraction.count()}):\n{datapd.Attraction.value_counts()}\n')
print(f'Title values:\n{datapd.Title.str.len().describe()}\n')
print(f'Opinion values:\n{datapd.Opinion.str.len().describe()}') 

Polarity values(38832):
5    20936
3     6363
4     5878
2     2920
1     2735
Name: Polarity, dtype: int64

Atraction values(38832):
Hotel         20058
Restaurant    11450
Attractive     7324
Name: Attraction, dtype: int64

Title values:
count    38831.000000
mean        25.978625
std         16.636964
min          1.000000
25%         15.000000
50%         22.000000
75%         33.000000
max        153.000000
Name: Title, dtype: float64

Opinion values:
count    38830.000000
mean       521.696343
std        861.882306
min         18.000000
25%        175.000000
50%        260.000000
75%        488.000000
max      20438.000000
Name: Opinion, dtype: float64


In [74]:
# filter non empty title
datapd = datapd[datapd.Title.str.len() > 0]
# filter non empty opinion
datapd = datapd[datapd.Opinion.str.len() > 0]



In [75]:
# filter by num of characters of Opinion
datapd = datapd[datapd.Opinion.str.len() < 5000]
# datapd.Opinion.str.len().sort_values()

In [76]:
for i in range(5):
    datapd.loc[datapd.Polarity == i+1, 'Polarity'] = i
    
datapd.loc[datapd.Attraction == "Hotel", 'Attraction'] = 0
datapd.loc[datapd.Attraction == "Restaurant", 'Attraction'] = 1
datapd.loc[datapd.Attraction == "Attractive", 'Attraction'] = 2


In [78]:
from datasets import Dataset
raw_dataset = Dataset.from_pandas(datapd)



In [83]:
raw_dataset5

AttributeError: 'Dataset' object has no attribute 'copy'

In [84]:
print(datapd)
raw_dataset1 = raw_dataset.shuffle(seed=50)
raw_datasets = raw_dataset1.train_test_split(0.2)
print(raw_datasets)
# raw_datasets.save_to_disk("./data")

                                       Title  \
0                               Pésimo lugar   
1                  No vayas a lugar de Eddie   
2               Mala relación calidad-precio   
3           Minusválido? ¡No te alojes aquí!   
4      Es una porqueria no pierdan su tiempo   
...                                      ...   
38827          Verdadera joya arquitectónica   
38828                              Romántico   
38829                     Parece un castillo   
38830                             Imperdible   
38831                       Muy bonita vista   

                                                 Opinion  Polarity Attraction  
0      Piensen dos veces antes de ir a este hotel, te...         0          0  
1      Cuatro de nosotros fuimos recientemente a Eddi...         0          1  
2      seguiré corta y simple: limpieza\n- bad. Tengo...         0          0  
3      Al reservar un hotel con multipropiedad Mayan ...         0          0  
4      No pierdan su ti

In [ ]:
from datasets import load_dataset, DatasetDict
raw_dataset = DatasetDict.load_from_disk("./data")

In [ ]:
raw_datasets.push_to_hub("javilonso/mex_data_augmented")


Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.
The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [87]:
1

1